In [1]:
import gmsh
gmsh.initialize()

In [2]:
# Data
l = 100
b = 10
es = 2

2

In [3]:
gmsh.model.add("rectangle")
rect = gmsh.model.occ.addRectangle(0, 0, 0, l, b, 0) # geometry: base x height
#gmsh.model.occ.mesh.setSize([2, rect], 20)
gmsh.model.occ.synchronize()

In [4]:
contact = gmsh.model.addPhysicalGroup(1, [2], 1)

1

In [5]:
ov = gmsh.model.getEntities(0);
gmsh.model.mesh.setSize(ov, es) # element size
gmsh.model.mesh.setAlgorithm(2, rect, 8)
gmsh.model.mesh.generate(1)
gmsh.model.mesh.generate(2)
gmsh.model.mesh.recombine()

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 50%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.00088272s, CPU 0.000875s)
Info    : 110 nodes 114 elements
Info    : Meshing 2D...
Info    : Meshing surface 0 (Plane, Frontal-Delaunay for Quads)
Info    : Done meshing 2D (Wall 0.088311s, CPU 0.064365s)
Info    : 306 nodes 614 elements
Info    : Recombining 2D mesh...
Info    : Blossom: 695 internal 106 closed
Info    : Blossom recombination completed (Wall 0.0140983s, CPU 0.011959s): 250 quads, 0 triangles, 0 invalid quads, 0 quads with Q < 0.1, avg Q = 0.998471, min Q = 0.974973
Info    : Done recombining 2D mesh (Wall 0.0141842s, CPU 0.012052s)


In [6]:
entities = gmsh.model.getEntities(2)
nodeTags, nodeCoords, nodeParams = gmsh.model.mesh.getNodes(-1, -1)
elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(2, -1)
#println(Int64.(elemNodeTags[1]))
#display(nodeCoords)
xynodes = reshape(nodeCoords, 3, :)
display(xynodes)
connectivity = (Int64.(reshape(elemNodeTags[1], 4, :)))'
display(connectivity)

3×306 Matrix{Float64}:
 0.0  100.0  100.0   0.0  2.0  4.0  6.0  …  20.0  14.0  8.0  50.0511   92.0
 0.0    0.0   10.0  10.0  0.0  0.0  0.0      6.0   6.0  6.0   4.07361   6.0
 0.0    0.0    0.0   0.0  0.0  0.0  0.0      0.0   0.0  0.0   0.0       0.0

250×4 adjoint(::Matrix{Int64}) with eltype Int64:
   1    3  145   57
   4  146  145    3
   6  143  196    5
   4    5  196  146
   2   58  143    6
   7    9  147  110
  10  144  147    9
  11  111  144   10
  11   12  197  111
  13  148  197   12
   ⋮            
 188  283  296  267
 186  282  298  269
 185  281  299  270
 184  280  300  271
 183  279  301  272
 182  278  302  273
 181  277  303  274
 180  276  304  275
 235  134  220  297

In [7]:
cnodes, xyznodes = gmsh.model.mesh.getNodesForPhysicalGroup(1, 1)
display(convert.(Int, cnodes))
display(reshape(xyznodes, 3, :))

6-element Vector{Int64}:
 1
 2
 3
 4
 5
 6

3×6 Matrix{Float64}:
 100.0  100.0  100.0  100.0  100.0  100.0
   0.0   10.0    2.0    4.0    6.0    8.0
   0.0    0.0    0.0    0.0    0.0    0.0

In [8]:
gmsh.model.getEntitiesForPhysicalGroup(-1, 1)

1-element Vector{Int32}:
 2

```julia
    gmsh.view.addModelData(tag, step, modelName, dataType, tags, data, time = 0., numComponents = -1, partition = 0)

Add model-based post-processing data to the view with tag `tag`. `modelName`
identifies the model the data is attached to. `dataType` specifies the type of
data, currently either "NodeData", "ElementData" or "ElementNodeData". `step`
specifies the identifier (>= 0) of the data in a sequence. `tags` gives the tags
of the nodes or elements in the mesh to which the data is associated. `data` is
a vector of the same length as `tags`: each entry is the vector of double
precision numbers representing the data associated with the corresponding tag.
The optional `time` argument associate a time value with the data.
`numComponents` gives the number of data components (1 for scalar data, 3 for
vector data, etc.) per entity; if negative, it is automatically inferred (when
possible) from the input data. `partition` allows one to specify data in several
sub-sets.

Types:
 - `tag`: integer
 - `step`: integer
 - `modelName`: string
 - `dataType`: string
 - `tags`: vector of sizes
 - `data`: vector of vectors of doubles
 - `time`: double
 - `numComponents`: integer
 - `partition`: integer
```

```julia
import gmsh
import sys

gmsh.initialize(sys.argv)

# Copied from discrete.py...
gmsh.model.add("test")
gmsh.model.addDiscreteEntity(2, 1)
gmsh.model.mesh.addNodes(2, 1, [1, 2, 3, 4],
                         [0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0.])
gmsh.model.mesh.addElements(2, 1, [2], [[1, 2]], [[1, 2, 3, 1, 3, 4]])
# ... end of copy

# Create a new post-processing view
t = gmsh.view.add("some data")

# add 10 steps of model-based data, on the nodes of the mesh
for step in range(0, 10):
    gmsh.view.addModelData(
        t,
        step,
        "test",
        "NodeData",
        [1, 2, 3, 4],  # tags of nodes
        [[10.], [10.], [12. + step], [13. + step]])  # data, per node

gmsh.view.write(t, "data.msh")

gmsh.finalize()
```

In [9]:
t = gmsh.view.add("test")
u = [zeros(3) for i in 1:306]
for j ∈ 0:100
    for i ∈ 1:306
        u[i] = [sin(j / 10 * xynodes[1, i] / l), sin(j / 10 * xynodes[1, i] / b), 0]
    end
    gmsh.view.addModelData(t, j, "rectangle", "NodeData", nodeTags, u, j * 0.1)
end

In [10]:
t2 = gmsh.view.add("tensor")
s = [zeros(9) for i in 1:306]
for i ∈ 1:306
    s[i] = [0, 1, 0, 1, 0, 0, 0, 0, 0]
end
gmsh.view.addModelData(t2, 0, "rectangle", "NodeData", nodeTags, s)

In [11]:
gmsh.option.setNumber("Mesh.NodeLabels", 0)

gmsh.fltk.run()
gmsh.finalize()

-------------------------------------------------------
Version       : 4.11.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 20221221
Build host    : gmsh.info
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blas[petsc] Blossom Cgns DIntegration Dlopen DomHex Eigen[contrib] Fltk Gmm[contrib] Hxt Jpeg Kbipack Lapack[petsc] LinuxJoystick MathEx[contrib] Med Mesh Metis[contrib] Mmg Mpeg Netgen ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom PETSc Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.4.0
PETSc version : 3.14.4 (real arithmtic)
OCC version   : 7.6.1
MED version   : 4.1.0
Packaged by   : geuzaine
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------
